<a href="https://colab.research.google.com/github/AnanmayS/BreastCancerPredictionNN/blob/main/BreastCancerPredictionipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Libs

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Running On GPU

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Current Device Used: {device}")

Current Device Used: cuda


Collecting Data and Preproccessing

In [ ]:
# Load dataset
data = load_breast_cancer()
X = data.data
y = data.target

Splitting Dataset into train and test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Standardize Data With Standard Scaler

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Convert train and test to tensors from numpy and move to GPU

X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test, dtype=torch.float32).to(device)

Creating NN Architecture

In [ ]:
class NeuralNetwork(nn.Module):

  def __init__(self, input_size, hidden_size, output_size):
    super(NeuralNetwork, self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, output_size)
    self.sigmoid = nn.Sigmoid()

  # forward prop
  def forward(self, x):
    output = self.fc1(x)
    output = self.relu(output)
    output = self.fc2(output)
    output = self.sigmoid(output)
    return output



In [ ]:
# hyperparamters

input_size = X_train.shape[1]
hidden_size = 64
output_size = 1
learn_rate = 0.001
epochs = 100

In [ ]:
# init model / move to gpu

model = NeuralNetwork(input_size, hidden_size, output_size).to(device)


In [ ]:
# Loss / Optimize

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learn_rate)

training NN

In [ ]:
for epoch in range(epochs):
    model.train()  # Set the model to training mode
    optimizer.zero_grad()  # Reset gradients to zero

    outputs = model(X_train)  # Forward pass
    loss = criterion(outputs, y_train.view(-1, 1))  # Calculate loss (fix typo)

    loss.backward()  # Backpropagation
    optimizer.step()  # Update model parameters

    # Check accuracy (without gradient tracking)
    with torch.no_grad():
        predicted = outputs.round()  # Round the outputs to get binary predictions
        correct = (predicted == y_train.view(-1, 1)).float().sum()  # Compare with actual labels
        acc = correct / y_train.size(0)  # Calculate accuracy

    # Print status every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f"Current Epoch: {epoch + 1}/{epochs}")
        print(f"Loss: {loss.item():.4f}")
        print(f"Accuracy: {acc.item() * 100:.2f}%")
        print("\n")


Current Epoch: 10/100
Loss: 0.1005
Accuracy: 97.80%


Current Epoch: 20/100
Loss: 0.0929
Accuracy: 97.80%


Current Epoch: 30/100
Loss: 0.0865
Accuracy: 98.24%


Current Epoch: 40/100
Loss: 0.0810
Accuracy: 98.24%


Current Epoch: 50/100
Loss: 0.0762
Accuracy: 98.24%


Current Epoch: 60/100
Loss: 0.0721
Accuracy: 98.24%


Current Epoch: 70/100
Loss: 0.0684
Accuracy: 98.46%


Current Epoch: 80/100
Loss: 0.0652
Accuracy: 98.46%


Current Epoch: 90/100
Loss: 0.0622
Accuracy: 98.46%


Current Epoch: 100/100
Loss: 0.0596
Accuracy: 98.46%




Evaluating The Training Data

In [ ]:
model.eval()

with torch.no_grad():
  outputs = model(X_train)
  predicted = outputs.round()  # Round the outputs to get binary predictions
  correct = (predicted == y_train.view(-1, 1)).float().sum()  # Compare with actual labels
  acc = correct / y_train.size(0)  # Calculate accuracy
  print(f"Training Data Accuracy: {acc.item() * 100:.2f}%")

Training Data Accuracy: 98.46%


Evaluating The Test Data

In [ ]:
model.eval()

with torch.no_grad():
  outputs = model(X_test)
  predicted = outputs.round()  # Round the outputs to get binary predictions
  correct = (predicted == y_test.view(-1, 1)).float().sum()  # Compare with actual labels
  acc = correct / y_test.size(0)  # Calculate accuracy
  print(f"Training Test Accuracy: {acc.item() * 100:.2f}%")

Training Test Accuracy: 97.37%
